# Get Palindromic Primes

In [1]:
from time import time
from json import dumps
from os import rename
from numpy import array
from src import get_palindromes, file_chunks, isprime

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

CHUNKSIZE = 500_000_000
OFFSET = 2
MIN_ODD_LEN = 9
STRETCH_MODE = True

INPUT_FILE = 'pi_100b.txt'
TEMP_FILE = f'_temp{str(time()).replace(".","")}.json'
OUTPUT_FILE = 'results.json'

In [2]:
def run_it():
    n_iterations = 0
    n_palindromes = 0
    n_primes = 0
    for idx, size, chunk in file_chunks(INPUT_FILE,CHUNKSIZE, OFFSET, MIN_ODD_LEN):
        odd_idx, odd_pld = get_palindromes(chunk, MIN_ODD_LEN, STRETCH_MODE)
        odd_primes  = [isprime(int(p)) for p in odd_pld]

        results  = [{'index':i+idx, 'palindrome': p, 'length': len(p), 'type':'odd' , 'prime': b} for i, p, b in zip(odd_idx, odd_pld, odd_primes)]

        with open(TEMP_FILE, 'a') as outF:
            for r in results:
                if r['prime']:
                    outF.write(dumps(r) + '\n')

        n_iterations += 1
        n_palindromes += len(odd_pld)
        n_primes += len([p for p in odd_primes if p])

        progress = (idx + CHUNKSIZE) / size
        if progress > 1: progress = 1
        print("\r({:6.2%}) {:.1E}/{:.1E} File: ...{} | Palindromes: {:5,d} | Primes: {:5,d}".format(
            progress,
            idx + CHUNKSIZE,
            size,
            INPUT_FILE[-20:],
            n_palindromes,
            n_primes,
        ), end='')

    rename(TEMP_FILE, OUTPUT_FILE)

In [3]:
run_it()

(100.00%) 1.0E+11/1.0E+11 File: ...pi_100b.txt | Palindromes: 11,108,665 | Primes: 562,123

# Calculate Results

In [4]:
from json import loads

first = []
counts = {}
actual_length = 0
largest_value = 0
largest_index = 0
with open('results.json', 'r') as f:
    for line in f:
        line = loads(line)
        counts[line['length']] = counts.get(line['length'], 0) + 1

        if line['length'] > actual_length:
            actual_length = line['length']
            first.append({'length': line['length'], 'index':line['index'], 'palindrome':line['palindrome']})
            largest_value = int(line['palindrome'])
            largest_index = line['index']
        
        elif line['length'] < actual_length and line['length'] not in [k['length'] for k in first]:
            first.append({'length': line['length'], 'index':line['index'], 'palindrome':line['palindrome']})

        elif line['length'] == actual_length:
            v = int(line['palindrome'])
            if v > largest_value:
                largest_value = v
                largest_index = line['index']

In [5]:
first

[{'length': 11, 'index': 5793498, 'palindrome': '74670707647'},
 {'length': 13, 'index': 25803984, 'palindrome': '1020776770201'},
 {'length': 15, 'index': 298503034, 'palindrome': '176860696068671'},
 {'length': 9, 'index': 129080, 'palindrome': '318272813'},
 {'length': 17, 'index': 6604858610, 'palindrome': '30948834143884903'},
 {'length': 19, 'index': 72075707768, 'palindrome': '7240428184818240427'}]

In [6]:
counts

{11: 41828, 13: 3524, 15: 305, 9: 516435, 17: 29, 19: 2}

In [7]:
largest_index, largest_value

(78833628391, 9125010550550105219)